In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import datetime
import matplotlib.pyplot as plt
import contextily as cx

## 读取数据 #df

In [3]:
path = 'process data/dataset2/'
df = pd.read_csv(path+'dataset_2.csv', usecols=[0,3,4])
df

,device_id,device_home_latitude,device_home_longitude
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036
1,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
3,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
4,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
...,...,...,...
59839133,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321534,114.260255
59839134,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321534,114.260255
59839135,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321534,114.260255
59839136,b4797889-7ba8-6302-2099-7fd7d0cf6e40,22.266985,114.234298


In [4]:
df = df.drop_duplicates()
df

,device_id,device_home_latitude,device_home_longitude
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
10,f68a046d-48d6-41bc-a8dc-8210b026d2a7,22.407392,114.224594
14,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,22.324517,114.160602
15,e9872505-727b-42f7-b4d5-efe02dc948d7,22.336465,114.245417
...,...,...,...
54706209,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,22.367530,114.127846
55123313,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,22.469127,114.006127
55997466,a4158a52-3c61-4f02-abbb-f035faa6faee,22.257824,114.165734
56447442,eebeebff-812f-4be2-aaa4-94f86c528427,22.290000,114.150000


In [ ]:
duplicates = df.duplicated(subset='device_id')
df[duplicates]
## 无重复deviceid

,device_id,device_home_latitude,device_home_longitude


In [ ]:
device_cnt = np.size(df['device_id'].unique())
device_cnt
## 有住址经纬度的用户有 105037 个

105037

In [ ]:
# ## 没有重复值，保存数据：
# df.to_csv(path + "devics_lat_lon.csv", index=False)

## 将 device经纬度 转换为 GeoDataFrame #device_lat_lon_gdf

In [5]:
geometry = gpd.points_from_xy(df['device_home_longitude'], df['device_home_latitude'])
device_lat_lon_gdf = gpd.GeoDataFrame(df, geometry=geometry)
device_lat_lon_gdf.crs = 'EPSG:4326'
device_lat_lon_gdf

,device_id,device_home_latitude,device_home_longitude,geometry
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036,POINT (114.25804 22.37802)
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828,POINT (114.13683 22.36588)
10,f68a046d-48d6-41bc-a8dc-8210b026d2a7,22.407392,114.224594,POINT (114.22459 22.40739)
14,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,22.324517,114.160602,POINT (114.16060 22.32452)
15,e9872505-727b-42f7-b4d5-efe02dc948d7,22.336465,114.245417,POINT (114.24542 22.33646)
...,...,...,...,...
54706209,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,22.367530,114.127846,POINT (114.12785 22.36753)
55123313,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,22.469127,114.006127,POINT (114.00613 22.46913)
55997466,a4158a52-3c61-4f02-abbb-f035faa6faee,22.257824,114.165734,POINT (114.16573 22.25782)
56447442,eebeebff-812f-4be2-aaa4-94f86c528427,22.290000,114.150000,POINT (114.15000 22.29000)


In [6]:
device_lat_lon_gdf = device_lat_lon_gdf.drop(['device_home_latitude','device_home_longitude'], axis=1)
device_lat_lon_gdf

,device_id,geometry
0,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
2,f6580b80-1451-48b9-b99f-685c17a54fc8,POINT (114.13683 22.36588)
10,f68a046d-48d6-41bc-a8dc-8210b026d2a7,POINT (114.22459 22.40739)
14,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,POINT (114.16060 22.32452)
15,e9872505-727b-42f7-b4d5-efe02dc948d7,POINT (114.24542 22.33646)
...,...,...
54706209,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,POINT (114.12785 22.36753)
55123313,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,POINT (114.00613 22.46913)
55997466,a4158a52-3c61-4f02-abbb-f035faa6faee,POINT (114.16573 22.25782)
56447442,eebeebff-812f-4be2-aaa4-94f86c528427,POINT (114.15000 22.29000)


## 读取building shape file #building_shp

In [34]:
import fiona

structure_path = 'Building_SHP/BUILDING_STRUCTURE.shp'
with fiona.open(structure_path) as f:
    shapefile = gpd.GeoDataFrame.from_features(f, crs=f.crs)
shapefile.to_crs(epsg=4326, inplace=True)
shapefile

,geometry,OBJECTID,BUILDINGST,BUILDINGCS,BUILDING00,CATEGORY,STATUS,STATUSDATE,OFFICIALBU,OFFICIAL00,NUMABOVEGR,NUMBASEMEN,TOPHEIGHT,BASEHEIGHT,GROSSFLOOR,RECORDCREA,RECORDUPDA,SHAPE_Leng,SHAPE_Area
0,"POLYGON ((113.84091 22.21870, 113.84090 22.218...",1,243561,0162608928T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,17.431311,14.453672
1,"POLYGON ((113.84094 22.21863, 113.84095 22.218...",2,243562,0163008922T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,12.960214,9.240269
2,"POLYGON ((113.84090 22.21867, 113.84091 22.218...",3,243563,0163108931T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,22.352643,31.015898
3,"POLYGON ((113.84097 22.21871, 113.84101 22.218...",4,243564,0163608930T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,18.595424,21.319706
4,"POLYGON ((113.84102 22.21870, 113.84101 22.218...",5,243565,0163808933T20071224,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20071224,20080116,19.609693,20.145564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340747,"POLYGON ((114.43641 22.53999, 114.43638 22.539...",340748,403798,6296144493T20190806,T,5,A,None,None,None,NaN,NaN,NaN,None,NaN,20190806,20190829,25.017263,39.012734
340748,"POLYGON ((114.43640 22.54116, 114.43640 22.541...",340749,4963,6296144621T20050430,T,9,A,None,None,None,NaN,NaN,10.1,None,NaN,20050430,20050725,34.926430,66.714851
340749,"POLYGON ((114.43668 22.54119, 114.43659 22.541...",340750,4952,6299144635T20050430,T,9,A,None,None,天后宮,NaN,NaN,9.7,None,NaN,20050430,20050725,43.341399,117.309442
340750,"POLYGON ((114.43675 22.54050, 114.43670 22.540...",340751,5040,6299744554T20050430,T,9,A,None,None,None,NaN,NaN,17.5,None,NaN,20050430,20050725,22.022560,29.099489


In [35]:
building_shp = shapefile[shapefile['BUILDING00'] == 'T']
building_shp = building_shp[['BUILDINGST', 'geometry']]
building_shp

,BUILDINGST,geometry
0,243561,"POLYGON ((113.84091 22.21870, 113.84090 22.218..."
1,243562,"POLYGON ((113.84094 22.21863, 113.84095 22.218..."
2,243563,"POLYGON ((113.84090 22.21867, 113.84091 22.218..."
3,243564,"POLYGON ((113.84097 22.21871, 113.84101 22.218..."
4,243565,"POLYGON ((113.84102 22.21870, 113.84101 22.218..."
...,...,...
340747,403798,"POLYGON ((114.43641 22.53999, 114.43638 22.539..."
340748,4963,"POLYGON ((114.43640 22.54116, 114.43640 22.541..."
340749,4952,"POLYGON ((114.43668 22.54119, 114.43659 22.541..."
340750,5040,"POLYGON ((114.43675 22.54050, 114.43670 22.540..."


In [36]:
# 增加缓冲区
building_shp['geometry'] = building_shp.geometry.buffer(distance=0.0001)
building_shp

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18108\2983928035.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  building_shp['geometry'] = building_shp.geometry.buffer(distance=0.0001)


,BUILDINGST,geometry
0,243561,"POLYGON ((113.84081 22.21859, 113.84080 22.218..."
1,243562,"POLYGON ((113.84093 22.21873, 113.84094 22.218..."
2,243563,"POLYGON ((113.84090 22.21857, 113.84089 22.218..."
3,243564,"POLYGON ((113.84087 22.21873, 113.84087 22.218..."
4,243565,"POLYGON ((113.84100 22.21861, 113.84099 22.218..."
...,...,...
340747,403798,"POLYGON ((114.43645 22.54009, 114.43646 22.540..."
340748,4963,"POLYGON ((114.43643 22.54126, 114.43644 22.541..."
340749,4952,"POLYGON ((114.43677 22.54114, 114.43676 22.541..."
340750,5040,"POLYGON ((114.43675 22.54040, 114.43670 22.540..."


In [11]:
import folium
# https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson.html
import geopandas as gpd

# 创建地图中心点坐标
center = [22.3193, 114.1694]  # 香港的纬度和经度

# 创建地图对象
m = folium.Map(location=center, zoom_start=12)

# 将建筑物图层添加到地图上
folium.GeoJson(building_shp[:1000]).add_to(m)

# # 保存地图为HTML文件
# m.save("map.html")

m

## 关联building_shp和device_lat_lon_gdf #merged

In [37]:
merged = gpd.sjoin(building_shp, device_lat_lon_gdf, how='inner', predicate='intersects')
# merged = merged.drop('index_right', axis=1)
merged ## 19435 rows

,BUILDINGST,geometry,index_right,device_id
116,192786,"POLYGON ((113.85569 22.25378, 113.85571 22.253...",2595202,814b8c7a-5ee1-414a-a1a9-39249d080f98
309,196195,"POLYGON ((113.86054 22.25505, 113.86053 22.255...",832268,f1e6291b-fe99-4a10-a740-9ef529b0abaa
310,246850,"POLYGON ((113.85967 22.25511, 113.85967 22.255...",832268,f1e6291b-fe99-4a10-a740-9ef529b0abaa
310,246850,"POLYGON ((113.85967 22.25511, 113.85967 22.255...",11403822,9d84fd00-873a-4473-9c17-42bbf0402455
331,185497,"POLYGON ((113.86075 22.25491, 113.86075 22.254...",11403822,9d84fd00-873a-4473-9c17-42bbf0402455
...,...,...,...,...
339981,41770,"POLYGON ((114.35908 22.46983, 114.35908 22.469...",104542,56de72b3-78d9-4aef-9b2a-3ed823245e4c
339986,41729,"POLYGON ((114.35898 22.46980, 114.35897 22.469...",104542,56de72b3-78d9-4aef-9b2a-3ed823245e4c
339992,265834,"POLYGON ((114.35890 22.46990, 114.35890 22.469...",104542,56de72b3-78d9-4aef-9b2a-3ed823245e4c
339995,41713,"POLYGON ((114.35907 22.47007, 114.35908 22.470...",104542,56de72b3-78d9-4aef-9b2a-3ed823245e4c


In [43]:
## 由于扩大了building shape，会有一个device映射到多个building的情况，取每个device出现最多次数的building为home building。

device_unproc = merged[['device_id','BUILDINGST']]
## 计算每个 device_id 的每个 BUILDINGST 出现的次数
counts = device_unproc.groupby(['device_id','BUILDINGST']).size()
## 保存 device_id 和 这个BUILDINGST出现次数最多的 BUILDINGST
max_BUILDINGST = counts.groupby('device_id').idxmax().apply(lambda x: x[1])
device_proc = pd.DataFrame({'device_id': max_BUILDINGST.index, 'BUILDINGST': max_BUILDINGST.values})
device_proc

## 105037 个有住址的用户中， 可以获得 住宅类building 的有 62913 个

,device_id,BUILDINGST
0,00000f8e-3fca-60b8-10b1-d27413b0cf19,82628
1,0000b42b-03f1-6be9-bf91-df48dfebb628,163805
2,000110af-768c-6abc-1068-8f00a49ab36a,183305
3,0001268a-33e0-6602-af37-799102373cfa,46381
4,0001ca98-70ec-6e99-be41-e272d9639921,144751
...,...,...
62908,fff54db6-88fe-41e7-aed1-5285466736aa,137098
62909,fff7e529-4fbf-443b-a898-5d5b46a0538a,165884
62910,fff894d6-9c6f-4bd2-92f3-77c135c96069,156165
62911,fffa70b5-2607-44de-a708-57a945ef9099,155936


In [52]:
## 保存数据：device_building_home
device_proc.to_csv(path + "device_building_home.csv", index=False)

In [49]:
unmapped = gpd.sjoin(building_shp, device_lat_lon_gdf, how='right', predicate='intersects')
unmapped

,index_left,BUILDINGST,device_id,geometry
0,321481.0,107993.0,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
0,321520.0,107930.0,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
0,321571.0,105126.0,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
0,321613.0,105082.0,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
2,191320.0,139639.0,f6580b80-1451-48b9-b99f-685c17a54fc8,POINT (114.13683 22.36588)
...,...,...,...,...
54706209,NaN,NaN,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,POINT (114.12785 22.36753)
55123313,NaN,NaN,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,POINT (114.00613 22.46913)
55997466,NaN,NaN,a4158a52-3c61-4f02-abbb-f035faa6faee,POINT (114.16573 22.25782)
56447442,NaN,NaN,eebeebff-812f-4be2-aaa4-94f86c528427,POINT (114.15000 22.29000)


In [50]:
## 105037 个有住址的用户中， 42124个用户无法映射到 building
unmapped = unmapped[unmapped['index_left'].isna()]
unmapped

,index_left,BUILDINGST,device_id,geometry
18,NaN,NaN,e99a8a9e-712e-4e15-9e59-cecfe7142aa4,POINT (114.15255 22.27501)
19,NaN,NaN,e9a3aeb7-6465-4aee-9aab-2f59ecb3b050,POINT (114.21819 22.38128)
24,NaN,NaN,f4465a0d-52ff-4916-91c7-4a759baf4749,POINT (114.19505 22.33237)
40,NaN,NaN,f4a37500-1401-4766-9be9-0b7492d7066f,POINT (114.21717 22.31672)
45,NaN,NaN,e8e2a152-0d6e-4af5-b1ee-5738c78078cf,POINT (114.25830 22.32406)
...,...,...,...,...
54706209,NaN,NaN,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,POINT (114.12785 22.36753)
55123313,NaN,NaN,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,POINT (114.00613 22.46913)
55997466,NaN,NaN,a4158a52-3c61-4f02-abbb-f035faa6faee,POINT (114.16573 22.25782)
56447442,NaN,NaN,eebeebff-812f-4be2-aaa4-94f86c528427,POINT (114.15000 22.29000)


In [51]:
import folium
# https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson.html
import geopandas as gpd

# 创建地图中心点坐标
center = [22.3193, 114.1694]  # 香港的纬度和经度

# 创建地图对象
m = folium.Map(location=center, zoom_start=12)

# 将建筑物图层添加到地图上
folium.GeoJson(unmapped[:1000]).add_to(m)

# # 保存地图为HTML文件
# m.save("map.html")

m